<a href="https://colab.research.google.com/github/kridtapon/MACD-Hull-Fusion-Strategy/blob/main/MACD_Hull_Fusion_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00
  Created wheel for vectorbt: filename=vectorbt-0.26.2-py3-none-any.whl size=527326 sha256=c6cc03c0420774fdad4f5e1d7184f04989fa081cc3073f5cec1578938fa0c1e0
  Stored in directory: /root/.cache/pip/wheels/35/a3/d2/ae1a8cbf659fffc7d1890123df1a7fa2e7f6df587c1dca71cd
Successfully built vectorbt
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
     

In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=812527f31ec112ee026c74c59786723af34848ef42c81feab2908dcc6863afb9
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import pandas as pd
import ta
import vectorbt as vbt
import numpy as np

def weighted_moving_average(series, period):
    """Calculate the Weighted Moving Average (WMA)."""
    weights = list(range(1, period + 1))
    return series.rolling(window=period).apply(lambda x: np.dot(x, weights) / sum(weights), raw=True)

def calculate_hull_average(df, period=14):
    """Calculate the Hull Moving Average (HMA)."""
    half_length = int(period / 2)
    sqrt_length = int(period ** 0.5)

    # Calculate WMA for half period and full period
    wma_half = weighted_moving_average(df['Close'], half_length)
    wma_full = weighted_moving_average(df['Close'], period)

    # Calculate the difference
    diff = 2 * wma_half - wma_full

    # Calculate HMA
    hull_avg = weighted_moving_average(diff, sqrt_length)
    return hull_avg

def main():
    # Load historical data
    df = pd.read_csv('XAUUSD_Candlestick_1_D_BID_01.01.2004-01.01.2024.csv')
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.ffill(inplace=True)

    # Calculate MACD and MACD histogram
    macd_indicator = ta.trend.MACD(close=df['Close'])
    df['MACD'] = macd_indicator.macd()
    df['MACD_Hist'] = macd_indicator.macd_diff()

    # Calculate 15-period Hull Moving Average
    df['Hull_Avg'] = calculate_hull_average(df, period=14)

    # Define entry signal
    df['Entry'] = (
        (df['MACD'] > 0) &
        (df['MACD_Hist'] > df['MACD_Hist'].shift(1)) &
        (df['MACD_Hist'].shift(1) < 0) &
        (df['Hull_Avg'] > df['Hull_Avg'].shift(1))
    )

    # Define exit signal
    df['Exit'] = False
    for i in range(1, len(df)):
        if df['Entry'].iloc[i]:
            for j in range(i + 1, min(i + 11, len(df))):
                if all(df['Close'].iloc[k] > df['Open'].iloc[k] for k in range(j - 2, j + 1)):
                    df['Exit'].iloc[j] = True
                    break

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Display performance metrics
    print(portfolio.stats())

    # Plot equity curve
    portfolio.plot().show()

if __name__ == "__main__":
    main()

<ipython-input-6-55f8f20843d6>:31: UserWarning:

Parsing dates in %d.%m.%Y %H:%M:%S.%f GMT%z format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

<ipython-input-6-55f8f20843d6>:57: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Start                         2003-12-31 05:00:00+07:00
End                           2024-01-01 05:00:00+07:00
Period                                             7307
Start Value                                    100000.0
End Value                                 441732.457761
Total Return [%]                             341.732458
Benchmark Return [%]                         396.560623
Max Gross Exposure [%]                            100.0
Total Fees Paid                            28183.044254
Max Drawdown [%]                              40.111523
Max Drawdown Duration                            3241.0
Total Trades                                         49
Total Closed Trades                                  48
Total Open Trades                                     1
Open Trade PnL                              7126.099147
Win Rate [%]                                  77.083333
Best Trade [%]                                37.522511
Worst Trade [%]                              -22